In [2]:
import pandas as pd
import numpy as np
# Read data from the csv file
review_df = pd.read_csv("data/故宫纪念品评价.csv")

In [3]:
print(review_df[:3])

     product_id              product_name sales     comment_id comment_time  \
0  550901982416  故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼  10万+  1222696402986   2023-11-03   
1  550901982416  故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼  10万+  1218267345769   2023-09-15   
2  550901982416  故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼  10万+  1218693191030   2023-09-15   

         content  sku  sku_id display_user_nick reply          create_time  \
0     此用户没有填写评价。  NaN     NaN              巧**慧   NaN  2023-11-03 14:59:16   
1           非常漂亮  NaN     NaN              c**9   NaN  2023-11-03 14:59:16   
2  很精致，买来做教师节礼物的  NaN     NaN              天**8   NaN  2023-11-03 14:59:16   

           update_time  
0  2023-11-03 14:59:16  
1  2023-11-03 14:59:16  
2  2023-11-03 14:59:16  


In [4]:
print(review_df['content'][:10])
review_df['content'] = review_df['content'].astype(str)

0                                           此用户没有填写评价。
1                                                 非常漂亮
2                                        很精致，买来做教师节礼物的
3                             很好看，但是送礼物了，没有打开看，希望朋友喜欢。
4                        质量很好，买的很合我心意，棒棒哒！✨需要的亲们可以放心购买
5                                           送给妹** 她很喜欢
6                                               除了贵没毛病
7                                               太好看了吧。
8      故宫博物院的文创及周边产品都很好，做工、质感、色泽，所以每次都是买了带去国外送给歪果仁，大爱。
9    宝贝非常好～～外包装很好，店家包装的很细心，很严实～～快递也很好～～三天就到了，快递员打电话...
Name: content, dtype: object


In [11]:
# 删除无效的评论
df = review_df[(review_df['content'] !=  "此用户没有填写评价。") & (review_df['content'] != None)]
print("处理后评价个数：", len(df), "处理前评价个数：", len(review_df))

# 评价改为字符串

df.loc[:, 'content'] = df['content'].astype(str)

处理后评价个数： 10209 处理前评价个数： 11035


In [15]:
df = df[['product_name', 'content', 'reply']] # 只保留商品名、评价、店家回复三栏
df.to_csv("data/故宫纪念品评价-清洗后.csv")
df

,product_name,content,reply
1,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,非常漂亮,NaN
2,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,很精致，买来做教师节礼物的,NaN
3,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,很好看，但是送礼物了，没有打开看，希望朋友喜欢。,NaN
4,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,质量很好，买的很合我心意，棒棒哒！✨需要的亲们可以放心购买,NaN
5,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,送给妹** 她很喜欢,NaN
...,...,...,...
11030,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,好,NaN
11031,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,漂亮,NaN
11032,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,故宫文创yyds！纸质非常好！,NaN
11033,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,好评,NaN


In [23]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
# 默认的BertTopic
import jieba
from stopwords import filter_stopwords
def tokenize_zh(text):
    words = jieba.lcut(text)
    return words

vectorizer = CountVectorizer(tokenizer=tokenize_zh, ngram_range=[1, 3])

import os
os.environ["https_proxy"] = "10.112.226.240:10811"
# secret_config = json.load(open("secret_config.json"))
# client = openai.OpenAI(api_key=secret_config["openai_key"], base_url=secret_config["openai_url"])
# from bertopic.representation import OpenAI
# representation_model = OpenAI(client, model="gpt-4", chat=True, prompt=prompt_cn)

from bertopic.representation import KeyBERTInspired #产生关键词
keybert_model = KeyBERTInspired()
representation_model = {
    "KeyBERT": keybert_model
}

prompt_cn = """
我有一个主题包含了以下的文档： 
[DOCUMENTS]
这个主题可以用以下的关键字描述： [KEYWORDS]

根据这些信息, 归纳一个整洁的主题，以如下格式返回:
topic: <topic label>
"""


topic_model = BERTopic(
    "chinese (simplified)", 
    hdbscan_model=HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True), 
    vectorizer_model=vectorizer, 
    representation_model=representation_model)

In [25]:
topics, probs = topic_model.fit_transform(df['content'])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.965 seconds.
Prefix dict has been built successfully.


In [26]:
df['bertopic'] = topics
df.to_csv('data/bertopic聚类结果.csv', header=False)

In [ ]:
topic_model.visualize_topics()

In [ ]:
for t in range(40):
    print(f"=================\n主题 {t}:", topic_model.get_topic(t)[0][0], f"占比：{np.sum(np.array(topics) == t)/len(topics) * 100:.1f}%")
    print(f"代表评价：")
    print("\n".join(topic_model.get_representative_docs(t)))

In [ ]:
topic_model.get_topic(3)

Total terms:  108938
删除项： 冲
删除项： 冲着
删除项： 冲着故
删除项： 冲着故宫买
删除项： 冲着故宫买的
删除项： 冲着故宫买的，
删除项： 冲着故宫买的，实
删除项： 着故
删除项： 着故宫买
删除项： 着故宫买的
删除项： 着故宫买的，
删除项： 着故宫买的，实
删除项： 着故宫买的，实物
删除项： 故
删除项： 故宫买
删除项： 故宫买的
删除项： 故宫买的，
删除项： 故宫买的，实
删除项： 故宫买的，实物
删除项： 故宫买的，实物不
删除项： 宫
删除项： 宫买
删除项： 宫买的
删除项： 宫买的，
删除项： 宫买的，实
删除项： 宫买的，实物
删除项： 宫买的，实物不
删除项： 宫买的，实物不咋
删除项： 买的，实
删除项： 买的，实物
删除项： 买的，实物不
删除项： 买的，实物不咋
删除项： 买的，实物不咋的
删除项： 的，实
删除项： 的，实物
删除项： 的，实物不
删除项： 的，实物不咋
删除项： 的，实物不咋的
删除项： 的，实物不咋的，
删除项： ，实物
删除项： ，实物不
删除项： ，实物不咋
删除项： ，实物不咋的
删除项： ，实物不咋的，
删除项： ，实物不咋的，价
删除项： 实物不
删除项： 实物不咋
删除项： 实物不咋的
删除项： 实物不咋的，
删除项： 实物不咋的，价
删除项： 实物不咋的，价格
删除项： 物不咋
删除项： 物不咋的
删除项： 物不咋的，
删除项： 物不咋的，价
删除项： 物不咋的，价格
删除项： 物不咋的，价格贵
删除项： 不咋
删除项： 不咋的
删除项： 不咋的，
删除项： 不咋的，价
删除项： 不咋的，价格
删除项： 不咋的，价格贵
删除项： 咋的
删除项： 咋的，
删除项： 咋的，价
删除项： 咋的，价格
删除项： 咋的，价格贵
删除项： 的，价
删除项： 的，价格
删除项： 的，价格贵
删除项： ，价
删除项： ，价格贵
删除项： 价格贵
删除项： 格
删除项： 格贵
删除项： 价格偏
删除项： 价格偏贵
删除项： 格偏
删除项： 格偏贵
删除项： 格偏贵了
删除项： 偏贵
删除项： 偏贵了
删除项： 不太值得
删除项： 太值
删除项： 太值得
删除项： 太值得买
删除项： 值得买
删除项： 得买
删除项： 不大，
删除项： 大，太
删除项： 设计
删除项： 设计还
删除项：

KeyboardInterrupt: 

In [17]:
terms = frequent_terms(["12345", "34512", "123"], 10)
print(terms)

{'1': 3, '12': 2, '123': 1, '1234': 1, '2': 2, '23': 1, '234': 1, '3': 2, '34': 2, '4': 2, '345': 1, '3451': 1, '45': 1, '451': 1, '5': 1, '51': 1}
删除项： 123
删除项： 2
删除项： 23
删除项： 234
删除项： 3
删除项： 4
删除项： 345
删除项： 45
删除项： 451
删除项： 5
删除项： 51
{'1': 3, '12': 2, '1234': 1, '34': 2, '3451': 1}


In [34]:
print(df[df['bertopic'] == 1]['content'])

39         冲着故宫买的，实物不咋的，价格贵了
44                    价格偏贵了。
59                    不太值得买。
62                     不大，太贵
63       设计还可以，就是这么小这个价钱有点贵了
                ...         
10967                 不太值得买。
10970                  不大，太贵
10971    设计还可以，就是这么小这个价钱有点贵了
10995     东西可以，就是又小又贵，纸张还可以。
11024    东西很好 但是还要收邮费有点让人不愉快
Name: content, Length: 458, dtype: object
